<a href="https://colab.research.google.com/github/maryantonopoulou/teliki_ergasia_EDDE2/blob/main/kathimerini_scraper_teasers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site και να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

In [ ]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_teaser_elements_from_articlesL(articlesL):
  articles_dataL = [] # κενή λίστα, στην οποία θα κάνουμε append() το dictionary με τα data κάθε άρθρου
  for article in articlesL:
    # title
    try:
      title = article.find("a", {"class": "py-4 mainlink"}).text.strip() # στην περίπτωση epikairothta
    except:
      try:
        title = article.find("p", class_=lambda c: c and "title is-5 medium_title" in c).text.strip() # στην περίπτωση κατηγοριών και αναζήτησης
      except:
        title=""
    # article url
    try:
      article_url = article.find("div", {"class": "content"}).a["href"] # στην περίπτωση κατηγοριών και αναζήτησης
    except:
      try:
        article_url = article.find("a", {"class": "py-4 mainlink"})["href"] # στην περίπτωση epikairothta
      except:
        article_url=""
    # date & time
    try:
      datetime_str = article.find("time")["datetime"] # στην περίπτωση epikairothta
      date_str = datetime_str.split("T")[0]
      time_str = datetime_str.split("T")[1].split("+")[0]
    except:
      try:
        datetime_str = article.find("div", {"class": "card-date"}).text.strip() # στην περίπτωση κατηγοριών και αναζήτησης
        dt_partsL = datetime_str.split(".")
        date_str = f"{dt_partsL[2]}-{dt_partsL[1]}-{dt_partsL[0]}"
        time_str = ""
      except:
        date_str = ""
        time_str = ""
    # section
    try:
      section = article.find("p", class_=lambda c: c and "subtitle" in c).text.strip()
    except:
      section = ""
    # excerpt
    try:
      excerpt = article.find("div", class_= lambda c: c and "lead" in c).text.strip()
    except:
      excerpt = ""
    # author
    try:
      author = article.find("span", {"class":"mb-2 author-link"}).text.strip()
    except:
      author = ""
    # Δημιουργία λεξικού με τα data του άρθρου
    articleD = {"title": title, "article_url": article_url, "date": date_str, "time": time_str, "section": section, "excerpt": excerpt, "author": author}

    # Προσθήκη των data του κάθε άρθρου στη λίστα των data των άρθρων
    articles_dataL.append(articleD)

  return articles_dataL

In [ ]:
url = "https://www.kathimerini.gr/search/%CF%84%CF%81%CE%BF%CF%87%CE%B1%CE%AF%CE%BF/page/1/?cat=%CE%9A%CE%BF%CE%B9%CE%BD%CF%89%CE%BD%CE%AF%CE%B1&usr&dt_from=01-01-2022&dt_to=31-12-2022"
print(len(url.split("/")))
url.split("/")

8


['https:',
 '',
 'www.kathimerini.gr',
 'search',
 '%CF%84%CF%81%CE%BF%CF%87%CE%B1%CE%AF%CE%BF',
 'page',
 '1',
 '?cat=%CE%9A%CE%BF%CE%B9%CE%BD%CF%89%CE%BD%CE%AF%CE%B1&usr&dt_from=01-01-2022&dt_to=31-12-2022']

In [ ]:
def scrape_kathimerini_teaser_pages(scrape_url, start_page_nbr=1, end_page_nbr=1):
  # Κενή αρχική λίστα
  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1): # βάζω +1 για να σκραπάρω και τον αριθμό του τελευταίου page που έχω ορίσει
    # η δομή των teasers pages urls του kathmerini.gr
    if "search" in scrape_url:
      search_term = scrape_url.split("/")[4]
      if len(scrape_url.split("/")) > 7:
        search_params = scrape_url.split("/")[-1]
      else:
        search_params = ""
      url = f'https://www.kathimerini.gr/search/{search_term}/page/{page_nbr}/{search_params}'
    else:
      category = scrape_url.split("/")[3]
      url = f"https://www.kathimerini.gr/{category}/page/{page_nbr}/"

    print(f"\rScraping page {page_nbr}: {url}" + " "*2, end="")

    # φέρνω τη σούπα της σελίδας με το function που έφτιαξα παραπάνω στο βήμα 2
    soup = get_webpage_soup(url)

    # βάζω τον κώδικα με τον οποίο βρίσκω όλα τα blocks των άρθρων - το έφτιαξα στο βήμα 6
    articlesL = soup.find_all("div", {"class":"nx-article"})

    # εξορύσσω τα δεδομένα όλων των άρθρων της σελίδας με το function που έφτιαξα στο βήμα 8
    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    # προσθέτω τα δεδομένα των άρθρων της σελίδας στη λίστα με τα δεδομένα όλων των άρθρων - τη δημιούργησα στην αρχή αυτού του function
    all_articles_dataL.extend(page_articles_dataL)

    # Τυχαία καθυστέρηση μεταξύ 1 και 3 δευτερολέπτων ανάμεσα στα page scrapes
    delay = random.uniform(1, 3)
    time.sleep(delay)

  # Δημιουργία dataframe από τη λίστα με τα λεξικά των δεδομένων κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")

  return df

In [ ]:
scrape_url=('https://www.kathimerini.gr/search/%CE%B5%CF%81%CE%B3%CE%B1%CF%84%CE%B9%CE%BA%CE%BF%20%CE%B1%CF%84%CF%85%CF%87%CE%B7%CE%BC%CE%B1/?cat&usr&dt_from&dt_to')
kathimerini_ergatika_atiximata_teasersdf = scrape_kathimerini_teaser_pages(scrape_url, 1,3)
kathimerini_ergatika_atiximata_teasersdf


Scraping page 1: https://www.kathimerini.gr/search/%CE%B5%CF%81%CE%B3%CE%B1%CF%84%CE%B9%CE%BA%CE%BF%20%CE%B1%CF%84%CF%85%CF%87%CE%B7%CE%BC%CE%B1/page/1/  https://www.kathimerini.gr/search/%CE%B5%CF%81%CE%B3%CE%B1%CF%84%CE%B9%CE%BA%CE%BF%20%CE%B1%CF%84%CF%85%CF%87%CE%B7%CE%BC%CE%B1/page/1/ was scraped successfully
Scraping page 2: https://www.kathimerini.gr/search/%CE%B5%CF%81%CE%B3%CE%B1%CF%84%CE%B9%CE%BA%CE%BF%20%CE%B1%CF%84%CF%85%CF%87%CE%B7%CE%BC%CE%B1/page/2/  https://www.kathimerini.gr/search/%CE%B5%CF%81%CE%B3%CE%B1%CF%84%CE%B9%CE%BA%CE%BF%20%CE%B1%CF%84%CF%85%CF%87%CE%B7%CE%BC%CE%B1/page/2/ was scraped successfully
Scraping page 3: https://www.kathimerini.gr/search/%CE%B5%CF%81%CE%B3%CE%B1%CF%84%CE%B9%CE%BA%CE%BF%20%CE%B1%CF%84%CF%85%CF%87%CE%B7%CE%BC%CE%B1/page/3/  https://www.kathimerini.gr/search/%CE%B5%CF%81%CE%B3%CE%B1%CF%84%CE%B9%CE%BA%CE%BF%20%CE%B1%CF%84%CF%85%CF%87%CE%B7%CE%BC%CE%B1/page/3/ was scraped successfully
Scraping completed. Fetshed 38 articles.


,title,article_url,date,time,section,excerpt,author
0,Θεσσαλονίκη: Εργατικό ατύχημα στη Σίνδο – Τραυ...,https://www.kathimerini.gr/society/563648845/t...,2025-06-04,,ΚΟΙΝΩΝΙΑ,Ο άνδρας μεταφέρθηκε στο «Παπανικολάου» σε σοβ...,
1,Θεσσαλονίκη: Σοβαρό εργατικό ατύχημα – Ι.Χ. έπ...,https://www.kathimerini.gr/society/563596888/t...,2025-05-06,,ΚΟΙΝΩΝΙΑ,Ενας εργαζόμενος σε έργα οδοποιίας τραυματίστη...,
2,Ενα εργατικό ατύχημα ανά δύο ημέρες το πρώτο τ...,https://www.kathimerini.gr/economy/563585770/e...,2025-04-29,,ΔΙΕΘΝΗΣ ΟΙΚΟΝΟΜΙΑ,Ραγδαία αύξηση παρουσιάζουν οι θάνατοι από εργ...,
3,Ηράκλειο: Εργατικό ατύχημα σε επιχείρηση με μά...,https://www.kathimerini.gr/society/563521963/i...,2025-03-19,,ΚΟΙΝΩΝΙΑ,"Ο 63χρονος ιδιοκτήτης συνελήφθη, ενώ η υπόθεση...",
4,Εργατικό ατύχημα στη Θεσσαλονίκη: Γυναίκα έπεσ...,https://www.kathimerini.gr/society/563478304/e...,2025-02-20,,ΚΟΙΝΩΝΙΑ,Η γυναίκα έχει τις αισθήσεις της ενώ έχει τραυ...,
5,Θεσσαλονίκη: Εργατικό ατύχημα στο λιμάνι με θύ...,https://www.kathimerini.gr/society/563395597/t...,2024-12-28,,ΚΟΙΝΩΝΙΑ,Ο τραυματίας διακομίστηκε με ασθενοφόρο του ΕΚ...,
6,ΓΣΕΕ: Νέο εργατικό ατύχημα στο λιμάνι του Πειραιά,https://www.kathimerini.gr/society/563183485/g...,2024-08-22,,ΚΟΙΝΩΝΙΑ,Η Συνομοσπονδία ζητά την άμεση παρέμβαση των α...,
7,Εργατικό ατύχημα στα Χανιά – Πιάστηκε το χέρι ...,https://www.kathimerini.gr/society/562879141/e...,2024-02-12,,ΚΟΙΝΩΝΙΑ,Η Πυροσβεστική έσπευσε στο σημείο με τρία οχήμ...,
8,Εργατικό ατύχημα στη Δραπετσώνα: Τραυματίστηκε...,https://www.kathimerini.gr/society/562800598/e...,2023-12-24,,ΚΟΙΝΩΝΙΑ,Το ατύχημα σημειώθηκε κατά τη διάρκεια πρόσδεσ...,
9,Σαλαμίνα: Εργατικό ατύχημα σε ναυπηγείο,https://www.kathimerini.gr/society/562648900/s...,2023-10-02,,ΚΟΙΝΩΝΙΑ,Ενας 52χρονος εργάτης τραυματίστηκε σήμερα σε ...,


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
kathimerini_ergatika_atiximata_teasersdf.to_csv('/content/drive/MyDrive/CSVFILES/kathimerini_ergatika_atiximata_teasersdf.csv')